# Chapter 15: Monoid, semigroup

## Semigroup exercises

In [1]:
:info Semigroup

class Semigroup a where
  (<>) :: a -> a -> a
  GHC.Base.sconcat :: GHC.Base.NonEmpty a -> a
  GHC.Base.stimes :: Integral b => b -> a -> a
  {-# MINIMAL (<>) #-}
  	-- Defined in ‘GHC.Base’
instance Semigroup Display -- Defined in ‘IHaskell.Types’
instance Semigroup (Either a b) -- Defined in ‘Data.Either’
instance Semigroup [a] -- Defined in ‘GHC.Base’
instance Semigroup Ordering -- Defined in ‘GHC.Base’
instance Semigroup a => Semigroup (Maybe a) -- Defined in ‘GHC.Base’
instance Semigroup a => Semigroup (IO a) -- Defined in ‘GHC.Base’
instance Semigroup b => Semigroup (a -> b) -- Defined in ‘GHC.Base’
instance (Semigroup a, Semigroup b, Semigroup c, Semigroup d, Semigroup e) => Semigroup (a, b, c, d, e) -- Defined in ‘GHC.Base’
instance (Semigroup a, Semigroup b, Semigroup c, Semigroup d) => Semigroup (a, b, c, d) -- Defined in ‘GHC.Base’
instance (Semigroup a, Semigroup b, Semigroup c) => Semigroup (a, b, c) -- Defined in ‘GHC.Base’
instance (Semigroup a, Semigroup b) => Semigroup (a, b) -- Defined in ‘GHC.Base’
instance Semigroup () -- Defined in ‘GHC.Base’

In [2]:
data Trivial = Trivial deriving (Eq, Show)

instance Semigroup Trivial where
    _ <> _ = Trivial

In [3]:
newtype Identity a = Identity a

instance Semigroup a => Semigroup (Identity a) where
    (Identity x) <> (Identity y) = Identity $ x <> y

In [4]:
data Two a b = Two a b

instance (Semigroup a, Semigroup b) => Semigroup (Two a b) where
    (Two x y) <> (Two x' y') = Two (x <> x') (y <> y')

In [5]:
data Three a b c = Three a b c

instance (Semigroup a, Semigroup b, Semigroup c) => Semigroup (Three a b c) where
    (Three x y z) <> (Three x' y' z') = Three (x <> x') (y <> y') (z <> z')

In [6]:
data Four a b c d = Four a b c d

instance (Semigroup a, Semigroup b, Semigroup c, Semigroup d) => Semigroup (Four a b c d) where
    (Four a b c d) <> (Four a' b' c' d') = Four (a <> a') (b <> b') (c <> c') (d <> d')

In [7]:
newtype BoolConj = BoolConj Bool

instance Semigroup BoolConj where
    (BoolConj x) <> (BoolConj y) = BoolConj $ x && y

In [8]:
newtype BoolDisj = BoolDisj Bool

instance Semigroup BoolDisj where
    (BoolDisj x) <> (BoolDisj y) = BoolDisj $ x || y

In [9]:
data Or a b
    = Fst a
    | Snd b
    
instance Semigroup (Or a b) where
    (Fst _) <> y = y
    x@(Snd _) <> _ = x

In [10]:
newtype Combine a b = Combine { unCombine :: a -> b }

instance Semigroup b => Semigroup (Combine a b) where
    (Combine f) <> (Combine g) = Combine $ \x -> f x <> g x

In [11]:
newtype Comp a = Comp { unComp :: a -> a }

instance Semigroup (Comp a) where
    (Comp f) <> (Comp g) = Comp $ f . g

In [12]:
data Validation a b
    = Failure a
    | Success b
    deriving (Eq, Show)
    
instance Semigroup a => Semigroup (Validation a b) where
    (Failure x) <> (Failure y) = Failure $ x <> y
    x@(Failure _) <> _ = x
    _ <> y = y

In [13]:
newtype AccumulateRight a b = AccumulateRight (Validation a b) deriving (Eq, Show)

instance Semigroup b => Semigroup (AccumulateRight a b) where
    (AccumulateRight (Success x)) <> (AccumulateRight (Success y)) = AccumulateRight $ Success $ x <> y
    x@(AccumulateRight (Failure _)) <> _ = x
    _ <> y = y

In [14]:
newtype AccumulateBoth a b = AccumulateBoth (Validation a b)
    deriving (Eq, Show)
    
instance (Semigroup a, Semigroup b) => Semigroup (AccumulateBoth a b) where
    (AccumulateBoth (Failure x)) <> (AccumulateBoth (Failure y)) = AccumulateBoth $ Failure $ x <> y
    (AccumulateBoth (Success x)) <> (AccumulateBoth (Success y)) = AccumulateBoth $ Success $ x <> y
    x@(AccumulateBoth (Failure _)) <> _ = x
    _ <> y = y

## Monoid exercises

In [15]:
instance Monoid Trivial where
    mempty = Trivial

In [16]:
instance Monoid a => Monoid (Identity a) where
    mempty = Identity mempty

In [17]:
instance (Monoid a, Monoid b) => Monoid (Two a b) where
    mempty = Two mempty mempty

In [18]:
instance Monoid BoolConj where
    mempty = BoolConj True

In [19]:
instance Monoid BoolDisj where
    mempty = BoolDisj False

In [20]:
instance Monoid b => Monoid (Combine a b) where
    mempty = Combine $ const mempty

In [21]:
instance Monoid (Comp a) where
    mempty = Comp id

In [22]:
newtype Mem s a = Mem { runMem :: s -> (a, s) }

instance Semigroup a => Semigroup (Mem s a) where
    (Mem f') <> (Mem f) = Mem $ fn where
        fn s = (a <> b, s'') where
            (a, s') = f s
            (b, s'') = f' s'

instance Monoid a => Monoid (Mem s a) where
    mempty = Mem $ \s -> (mempty, s)

f' :: Num s => Mem s String
f' = Mem $ \s -> ("hi", s + 1)

runMem (f' <> f') 0
runMem (f' <> mempty) 0
runMem (mempty <> f') 0
runMem mempty 0 :: (String, Int)
runMem (f' <> mempty) 0 == runMem f' 0
runMem (mempty <> f') 0 == runMem f' 0

("hihi",2)

("hi",1)

("hi",1)

("",0)

True

True